## 9.water shed

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

I = cv2.imread("/content/image25.jpeg")

# Check if the image is read correctly
if I is None:
    raise FileNotFoundError('Could not open or find the image.')

# Step 2: Convert to Grayscale (if necessary)
if len(I.shape) == 3:
    I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

# Step 3: Noise Reduction
I_filtered = cv2.GaussianBlur(I, (5, 5), 2)  # Apply Gaussian filter to reduce noise

# Step 4: Compute the Gradient Magnitude
sobelx = cv2.Sobel(I_filtered, cv2.CV_64F, 1, 0, ksize=3)
sobely = cv2.Sobel(I_filtered, cv2.CV_64F, 0, 1, ksize=3)
gradmag = np.sqrt(sobelx**2 + sobely**2)

# Step 5: Marker-Based Segmentation
_, binary_image = cv2.threshold(I_filtered, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
D = ndi.distance_transform_edt(binary_image)
local_max = cv2.dilate(cv2.erode(D, None), None)
markers, _ = ndi.label(local_max)
gradmag2 = cv2.watershed(cv2.cvtColor((gradmag * 255 / gradmag.max()).astype(np.uint8), cv2.COLOR_GRAY2BGR), markers)

# Step 6: Apply the Watershed Transform
labels = cv2.watershed(cv2.cvtColor(I_filtered, cv2.COLOR_GRAY2BGR), gradmag2)

# Step 7: Define 7 distinct colors for visualization
colors = [
    [255, 0, 0], [0, 255, 0], [0, 0, 255], [255, 255, 0],
    [0, 255, 255], [255, 0, 255], [192, 192, 192]
]

# Create an empty image for the result
Lrgb = np.zeros((*labels.shape, 3), dtype=np.uint8)

# Color the regions
for label in np.unique(labels):
    if label == -1:  # Ignore background
        continue
    mask = labels == label
    color_idx = (label - 1) % 7
    Lrgb[mask] = colors[color_idx]

# Step 8: Visualize the Results
plt.figure(figsize=(10, 10))

# Original Image
plt.subplot(2, 2, 1)
plt.imshow(I, cmap='gray')
plt.title('Original Image')
plt.axis('off')

# Gradient Magnitude
plt.subplot(2, 2, 2)
plt.imshow(gradmag, cmap='gray')
plt.title('Gradient Magnitude')
plt.axis('off')

# Binary Image
plt.subplot(2, 2, 3)
plt.imshow(binary_image, cmap='gray')
plt.title('Binary Image')
plt.axis('off')

# Watershed Transform
plt.subplot(2, 2, 4)
plt.imshow(Lrgb)
plt.title('Watershed Transform')
plt.axis('off')

plt.tight_layout()
plt.show()


## 11.CNN with and without pca

In [ ]:
# Import necessary libraries
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model

image_path = '/content/drive/MyDrive/de975595890f0ed79238dc4d61532777.jpg'

def extract_features_with_cnn(image_path):
    base_model = VGG16(weights='imagenet', include_top=False)
    # Extract features from an earlier layer for even higher resolution
    model = Model(inputs=base_model.inputs, outputs=base_model.get_layer('block2_pool').output)

    img = cv.imread(image_path)
    img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img_resized = cv.resize(img_rgb, (224, 224))
    img_array = np.expand_dims(img_resized, axis=0)
    img_preprocessed = preprocess_input(img_array)

    features = model.predict(img_preprocessed)
    features_reshaped = features.reshape(-1, features.shape[-1])

    return features_reshaped, img_rgb, features.shape[1:3]  # Return spatial dimensions

def kmeans_with_and_without_pca(image_path, cluster_n):
    features, img_rgb, feature_map_size = extract_features_with_cnn(image_path)

    # KMeans clustering without PCA
    kmeans_no_pca = KMeans(n_clusters=cluster_n, random_state=42)
    labels_no_pca = kmeans_no_pca.fit_predict(features)

    silhouette_no_pca = silhouette_score(features, labels_no_pca)
    print(f"Silhouette Score without PCA: {silhouette_no_pca:.4f}")

    # KMeans clustering with PCA
    pca = PCA(n_components=2)
    features_pca = pca.fit_transform(features)

    kmeans_pca = KMeans(n_clusters=cluster_n, random_state=42)
    labels_pca = kmeans_pca.fit_predict(features_pca)

    silhouette_pca = silhouette_score(features_pca, labels_pca)
    print(f"Silhouette Score with PCA: {silhouette_pca:.4f}")

    # Segmented image visualization (no PCA)
    segmented_img = labels_no_pca.reshape(feature_map_size)  # Reshape labels to feature map size

    # Resize segmented image back to original image size
    segmented_img_resized = cv.resize(segmented_img.astype(np.uint8), (img_rgb.shape[1], img_rgb.shape[0]), interpolation=cv.INTER_LINEAR)

    # Apply a bilateral filter for edge-aware smoothing (preserves edges)
    segmented_img_filtered = cv.bilateralFilter(segmented_img_resized, d=9, sigmaColor=75, sigmaSpace=75)

    # Plotting the results in a 2x2 grid with reduced size
    plt.figure(figsize=(10, 8))  # Reduced figsize

    # Original image
    plt.subplot(2, 2, 1)
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.title('Input Image')

    # PCA feature plot
    plt.subplot(2, 2, 2)
    scatter = plt.scatter(features_pca[:, 0], features_pca[:, 1], c=labels_pca, cmap='tab10', s=20)
    plt.colorbar(scatter, ticks=range(cluster_n), label='Cluster')
    plt.title('PCA of CNN-extracted Features (with K-means Clustering)')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')

    # Segmented image
    plt.subplot(2, 2, 3)
    plt.imshow(segmented_img_filtered, cmap='tab10')
    plt.axis('off')
    plt.title('Clearer Segmented Image with Smoothing')

    # Silhouette score comparison plot
    plt.subplot(2, 2, 4)
    scores = [silhouette_no_pca, silhouette_pca]
    labels = ['Without PCA', 'With PCA']
    plt.bar(labels, scores, color=['skyblue', 'salmon'])
    plt.ylabel('Silhouette Score')
    plt.title('Comparison of Silhouette Scores')
    plt.ylim(0, 1)

    plt.tight_layout()
    plt.show()

# Running the KMeans clustering with CNN features
cluster_n = 5
kmeans_with_and_without_pca(image_path, cluster_n)


## 4.edge detection

In [ ]:
import cv2
import numpy as np
import scipy.ndimage as nd
from scipy import signal
import matplotlib.pyplot as plt

# Load the original image
img = cv2.imread("/content/drive/MyDrive/cat.jpeg", 1)
if img is None:
    raise FileNotFoundError("The specified image path is incorrect or the image cannot be found.")
original_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Gaussian Blur
img_gaussian = cv2.GaussianBlur(gray, (3, 3), 0)

# Canny Edge Detection
img_canny = cv2.Canny(img, 100, 200)

# Sobel Edge Detection
img_sobelx = cv2.Sobel(img_gaussian, cv2.CV_8U, 1, 0, ksize=5)
img_sobely = cv2.Sobel(img_gaussian, cv2.CV_8U, 0, 1, ksize=5)
img_sobel = img_sobelx + img_sobely

# Prewitt Edge Detection
kernelx = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])
kernely = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
img_prewittx = cv2.filter2D(img_gaussian, -1, kernelx)
img_prewitty = cv2.filter2D(img_gaussian, -1, kernely)
img_prewitt = img_prewittx + img_prewitty

# Laplacian Edge Detection
laplacian = cv2.Laplacian(img_gaussian, cv2.CV_64F)

# LoG (Laplacian of Gaussian)
img2 = cv2.imread("/content/drive/MyDrive/cat.jpeg")
if img2 is None:
    raise FileNotFoundError("The specified image path is incorrect or the image cannot be found.")
gray_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
LoG = nd.gaussian_laplace(gray_img2, 2)
thres = np.absolute(LoG).mean() * 0.75
output = np.zeros(LoG.shape)

# Zero-crossing detection for LoG
w = output.shape[1]
h = output.shape[0]
for y in range(1, h - 1):
    for x in range(1, w - 1):
        patch = LoG[y - 1:y + 2, x - 1:x + 2]
        p = LoG[y, x]
        maxP = patch.max()
        minP = patch.min()
        if (p > 0).any():
            zeroCross = True if minP < 0 else False
        else:
            zeroCross = True if maxP > 0 else False
        if ((maxP - minP) > thres) and zeroCross:
            output[y, x] = 1

# DoG (Difference of Gaussian)
A = np.array([[0, 0, -1, -1, -1, 0, 0],
              [0, -2, -3, -3, -3, -2, 0],
              [-1, -3, 5, 5, 5, -3, -1],
              [-1, -3, 5, 16, 5, -3, -1],
              [-1, -3, 5, 5, 5, -3, -1],
              [0, -2, -3, -3, -3, -2, 0],
              [0, 0, -1, -1, -1, 0, 0]], dtype=np.float32)

ratio = img2.shape[0] / 500.0
new_width = int(img2.shape[1] / ratio)
original = img2.copy()
nimg = cv2.resize(gray, (new_width, 500))

I1 = signal.convolve2d(nimg, A)
I1 = np.absolute(I1)
I1 = (I1 - np.min(I1)) / float(np.max(I1) - np.min(I1))

I2 = cv2.GaussianBlur(I1, (5, 5), 0)

# Plotting the results
plt.figure(figsize=(20, 10))

plt.subplot(2, 4, 1), plt.imshow(original_img)
plt.title('Original Image'), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 2), plt.imshow(gray, cmap='gray')
plt.title('Gray Scale Image'), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 3), plt.imshow(img_canny, cmap='gray')
plt.title('Canny Edge Detection'), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 4), plt.imshow(img_sobel, cmap='gray')
plt.title('Sobel Edge Detection'), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 5), plt.imshow(img_prewitt, cmap='gray')
plt.title('Prewitt Edge Detection'), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 6), plt.imshow(laplacian, cmap='gray')
plt.title('Laplacian Edge Detection'), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 7), plt.imshow(output, cmap='gray')
plt.title('LoG Edge Detection'), plt.xticks([]), plt.yticks([])

plt.subplot(2, 4, 8), plt.imshow(I2, cmap='gray')
plt.title('DoG Edge Detection'), plt.xticks([]), plt.yticks([])

plt.show()


## 6.houghline transformation

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

img = cv2.imread("/content/geometric-shapes-clip-art-294298.png")

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Perform Canny edge detection
edges = cv2.Canny(gray, 50, 150)  # Adjusted threshold values

# Parameters for Hough transform
minLineLength = 3
maxLineGap = 1

# Perform Hough line detection
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 50, minLineLength, maxLineGap)  # Adjusted parameters

# Create a new figure
plt.figure(figsize=(12, 8))

# Display grayscale image
plt.subplot(2, 2, 1)
plt.imshow(gray, cmap='gray')
plt.title('Grayscale Image')
plt.axis('off')

# Display edge-detected image
plt.subplot(2, 2, 2)
plt.imshow(edges, cmap='gray')
plt.title('Edges')
plt.axis('off')

# Display the original image with Hough lines overlayed
plt.subplot(2, 2, (3, 4))
img_with_lines = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(img_with_lines, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw lines on the image
plt.imshow(img_with_lines)
plt.title('Hough Lines')
plt.axis('off')

plt.tight_layout()
plt.show()


## 8.brute force

In [ ]:
import cv2 as cv
from matplotlib import pyplot as plt

# File paths from your Google Drive
img1 = cv.imread('/content/drive/MyDrive/brute force image/eat the frog 1.jpg', cv.IMREAD_GRAYSCALE)
img2 = cv.imread('/content/drive/MyDrive/brute force image/eat the frog 2.jpg', cv.IMREAD_GRAYSCALE)

# Check if images loaded correctly
if img1 is None or img2 is None:
    print("Error: Could not load one or both images.")
else:
    # Print the OpenCV version
    print('OpenCV Version:', cv.__version__)

    # Initiate SIFT detector
    sift = cv.SIFT_create()

    # Find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)  # Or pass an empty dictionary

    flann = cv.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    # Create a mask to draw only good matches
    matchesMask = [[0, 0] for i in range(len(matches))]

    # Ratio test as per Lowe's paper
    for i, (m, n) in enumerate(matches):
        if m.distance < 0.7 * n.distance:
            matchesMask[i] = [1, 0]

    # Draw parameters for matches
    draw_params = dict(matchColor=(0, 255, 0),  # Draw good matches in green
                       singlePointColor=(255, 0, 0),  # Draw keypoints in red
                       matchesMask=matchesMask,  # Draw only matches that passed the ratio test
                       flags=cv.DrawMatchesFlags_DEFAULT)

    # Draw the matches
    img3 = cv.drawMatchesKnn(img1, kp1, img2, kp2, matches, None, **draw_params)

    # Display the matched image
    plt.imshow(img3)
    plt.show()


## 12.RNN and faster RCNN

In [ ]:
# Import necessary libraries
import torch
import torchvision
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.ops import box_iou
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from google.colab import files

model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

image_path = "/content/cat.jpeg"
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.imshow(image_rgb)
plt.axis('off')
plt.show()

transform = transforms.Compose([
    transforms.ToTensor()
])
image_tensor = transform(image_rgb).unsqueeze(0)


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
image_tensor = image_tensor.to(device)

ground_truth_boxes = torch.tensor([[50, 50, 200, 200]]).float().to(device)

with torch.no_grad():
    output = model(image_tensor)[0]

pred_boxes = output['boxes']
pred_scores = output['scores']

iou_threshold = 0.5
true_positives, false_positives, false_negatives = 0, 0, 0

if len(pred_boxes) > 0 and len(ground_truth_boxes) > 0:
    ious = box_iou(pred_boxes, ground_truth_boxes)

    for i, iou_row in enumerate(ious):
        max_iou, _ = iou_row.max(0)
        if max_iou >= iou_threshold:
            true_positives += 1
        else:
            false_positives += 1

    false_negatives += len(ground_truth_boxes) - len(ious)

else:
    false_negatives += len(ground_truth_boxes)

precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')

for box in pred_boxes:
    x1, y1, x2, y2 = map(int, box)
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

cv2_imshow(image)


## 10.kmeans

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Read the image using OpenCV
image = cv2.imread("/content/drive/MyDrive/cat.jpeg")

# Convert BGR to RGB
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Reshape the image to a 2D array of pixels (each pixel is a vector of 3 values: R, G, B)
pixels = image_rgb.reshape((-1, 3))
pixels = np.float32(pixels)

# Define criteria and apply KMeans
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
k = 4  # Number of clusters
_, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

# Convert centers to uint8 (color values)
centers = np.uint8(centers)

# Map labels to the corresponding centers (cluster centers)
segmented_image = centers[labels.flatten()]

# Reshape back to the original image size
segmented_image = segmented_image.reshape(image_rgb.shape)

# Calculate the number of pixels in each cluster
unique, counts = np.unique(labels, return_counts=True)

# Set up the figure for multiple subplots
fig = plt.figure(figsize=(18, 10))

# Plot original image
ax1 = fig.add_subplot(2, 3, 1)
ax1.imshow(image_rgb)
ax1.set_title('Original Image')
ax1.axis('off')

# Plot segmented image
ax2 = fig.add_subplot(2, 3, 2)
ax2.imshow(segmented_image)
ax2.set_title('Segmented Image (K={})'.format(k))
ax2.axis('off')

# Create and display a simple pivot-like table with pixel counts per cluster
ax_pivot = fig.add_subplot(2, 3, 3)
table_data = list(zip(unique, counts))
col_labels = ['Cluster', 'Pixel Count']

# Create a table in the plot
ax_pivot.axis('tight')
ax_pivot.axis('off')
table = ax_pivot.table(cellText=table_data, colLabels=col_labels, loc='center')
table.set_fontsize(12)
table.scale(1, 1.5)
ax_pivot.set_title("Pixel Count per Cluster")

# Plot the 3D scatter diagram for clustering in RGB space
ax3 = fig.add_subplot(2, 3, (4, 6), projection='3d')
r = pixels[:, 0]
g = pixels[:, 1]
b = pixels[:, 2]

# Use labels to color the pixels in the scatter plot
ax3.scatter(r, g, b, c=labels.flatten(), s=1, cmap='viridis')
ax3.set_xlabel('Red')
ax3.set_ylabel('Green')
ax3.set_zlabel('Blue')
ax3.set_title('3D RGB Clustering Plot (K={})'.format(k))

# Show the final plot
plt.tight_layout()
plt.show()


## 7.harris corner

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread("/content/OIP.jpeg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for matplotlib display

# Display original image
plt.imshow(img)
plt.title('Original Image')
plt.axis('off')
plt.show()

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
gray = np.float32(gray)

# Apply Harris Corner Detection
dst = cv2.cornerHarris(gray, 2, 3, 0.04)

# Result is dilated for marking the corners
dst = cv2.dilate(dst, None)

# Threshold for an optimal value, it may vary depending on the image
img[dst > 0.01 * dst.max()] = [255, 0, 0]

# Display result
plt.imshow(img)
plt.title('Harris Corner Detection')
plt.axis('off')
plt.show()


## 5.histogram computer vision

In [ ]:
import cv2
import matplotlib.pyplot as plt

image = cv2.imread('/cat.jpeg')

if image is None:
    print("Error: Unable to read the image file.")
else:
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    images = [gray_image, image]
    titles = ['Grayscale Image', 'Color Image']
    channels = ['Grayscale', 'Blue', 'Green', 'Red']
    colors = ['black', 'blue', 'green', 'red']

    plt.figure(figsize=(12, 6))

    for i in range(2):
        plt.subplot(2, 3, i + 1)
        if i == 0:
            plt.imshow(images[i], cmap='gray')
        else:
            plt.imshow(cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB))
        plt.title(titles[i])
        plt.axis('off')

    for j, (channel, color) in enumerate(zip(channels, colors)):
        plt.subplot(2, 3, j + 3)
        if channel == 'Grayscale':
            hist = cv2.calcHist([gray_image], [0], None, [256], [0, 256])
        else:
            hist = cv2.calcHist([image], [j - 1], None, [256], [0, 256])
        plt.plot(hist, color=color)
        plt.title(f'{channel} Histogram')
        plt.xlabel('Bins')
        plt.ylabel('# of Pixels')
        plt.xlim([0, 256])

    plt.tight_layout()
    plt.show()


## 3.morphology

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

# Read image
img = cv.imread("/content/cat.jpeg", 1)
kernel = np.ones((5, 5), np.uint8)

# Apply morphological operations
erosion = cv.erode(img, kernel, iterations=1)
dilation = cv.dilate(img, kernel, iterations=1)
opening = cv.morphologyEx(img, cv.MORPH_OPEN, kernel)
closing = cv.morphologyEx(img, cv.MORPH_CLOSE, kernel)
gradient = cv.morphologyEx(img, cv.MORPH_GRADIENT, kernel)
tophat = cv.morphologyEx(img, cv.MORPH_TOPHAT, kernel)
blackhat = cv.morphologyEx(img, cv.MORPH_BLACKHAT, kernel)

# Create a list of titles and corresponding images
titles = ['Original', 'Erosion', 'Dilation', 'Opening', 'Closing', 'Gradient', 'Tophat', 'Blackhat']
images = [img, erosion, dilation, opening, closing, gradient, tophat, blackhat]

# Plot all images in a single figure
plt.figure(figsize=(20, 15))
for i in range(len(images)):
    plt.subplot(2, 4, i + 1)  # 2 rows, 4 columns
    plt.imshow(cv.cvtColor(images[i], cv.COLOR_BGR2RGB))
    plt.title(titles[i])
    plt.axis('off')

# Display the plot
plt.tight_layout()
plt.show()


## grapcut